# Ewolucja "Alei Tornad" – Analiza Przestrzenno-Czasowa (1950-2015)

## 1. Przedmiot Analizy
Projekt skupia się na **analizie zmian geograficznego rozkładu tornad** w Stanach Zjednoczonych na przestrzeni ponad 60 lat. Tradycyjna wiedza meteorologiczna definiuje "Aleję Tornad" (Tornado Alley) jako obszar Wielkich Równin (Teksas, Oklahoma, Kansas). Jednak nowsze badania sugerują, że w wyniku zmian klimatycznych strefa największego ryzyka przesuwa się na wschód, w stronę gęściej zaludnionych stanów (tzw. "Dixie Alley" - Mississippi, Alabama, Tennessee).

W ramach projektu analizujemy:
* **Przesunięcie centroidu:** Jak zmieniał się "środek ciężkości" występowania tornad dekada po dekadzie.
* **Intensywność zjawisk:** Czy zmiany dotyczną tylko najsilniejszych tornad, czy może słabszych również.
* **Zmiany w lokalizacji tornad:** Jak faktycznie na przestrzeni lat zmienia się występowanie tornad.
* **Czynnik dobowy:** Różnice w ofiarach i rannych względem tornad dziennych i nocnych.

## 2. Cel Projektu
Głównym celem jest weryfikacja hipotezy o przesunięciu strefy zagrożenia na wschód.

Cele szczegółowe obejmują:
* **Wizualizację trendów:** Stworzenie interaktywnych map pozwalających zrozumieć dynamikę zmian w czasie.
* **Identyfikację nowych stref ryzyka:** Wskazanie obszarów, które historycznie były bezpieczne, a obecnie notują wzrost aktywności tornad.
* **Analizę bezpieczeństwa:** Czy obywatele wschodnich stanów powinni być przygotowani na tornada niewystępujące wcześniej w ich regionie.
* **Zastosowanie praktyczne:** Wyniki mogą być użyteczne dla sektora ubezpieczeń (ocena ryzyka) oraz planowania urbanistycznego (wymogi budowlane).

---

## 3. Opis Zbioru Danych
* **Dane pochodzą z:** NOAA's National Weather Service Storm Prediction Center (SPC) i obejmują wszystkie udokumentowane tornada w latach 1950–2015.

### Kluczowe atrybuty (Kolumny):

| Nazwa Kolumny                 | Typ     | Opis                                                                                         |
|:------------------------------|:--------|:---------------------------------------------------------------------------------------------|
| **Identyfikacja i Czas**      |
| `om`                          | Integer | ID tornada .                                                                                 |
| `yr`, `mo`, `dy`              | Integer | Rok, Miesiąc, Dzień wystąpienia zjawiska.                                                    |
| `date`                        | Date    | Pełna data (format YYYY-MM-DD).                                                              |
| `time`                        | Time    | Godzina wystąpienia zjawiska (HH:MM:SS).                                                     |
| `tz`                          | Integer | Strefa czasowa.                                                                              |
| **Lokalizacja Geograficzna**  |
| `state`                       | String  | Kod dwuliterowy stanu (np. TX, OK, AL).                                                      |
| `state_full`                  | String  | Pełna nazwa stanu.                                                                           |
| `slat` / `slon`               | Float   | Start Latitude/Longitude. Współrzędne początkowe tornada.                                    |
| `elat` / `elon`               | Float   | End Latitude/Longitude. Współrzędne końcowe tornada.                                         |
| **Parametry Meteorologiczne** |
| `mag`                         | Integer | Magnituda (siła) w skali F (Fujita). Wartości od 0 (najsłabsze) do 5 (katastrofalne).        |
| **Skutki i Straty**           |
| `fat`                         | Integer | Liczba ofiar śmiertelnych.                                                                   |
| `inj`                         | Integer | Liczba rannych.                                                                              |
| `loss`                        | Float   | Szacowane straty majątkowe. - możemy jeszcze spróbować dodać do jakiś mapek jakbyśmy chcieli |

### Uwagi do danych:
1.  **Szum danych (Population Bias):** W ostatnich dekadach notuje się znacznie więcej słabych tornad (F0/F1) ze względu na lepszą technologię (radary dopplerowskie) i smartfony. Aby zachować rzetelność analizy trendów wieloletnich, projekt skupia się głównie na tornadach silnych (**F1+** lub **F3+**), których wykrywalność w latach 50. była zbliżona do dzisiejszej.
2.  **Skala:** W 2007 roku USA przeszły ze skali Fujity (F) na Ulepszoną Skalę Fujity (EF). W zbiorze są one traktowane ciągle (jako `mag`).
3.  **Pominięte dane:** W opisie danych pominięte zostały dane, które nie zostały wykorzystane w analizie przestrzennej. Pełny zbiór danych zawiera 35 kolumn.
4.  **Dodane ręcznie dane pomocnicze:** Dla wizualizacji danych dla stanów został dodany słownik mapujący kody stanów ('state') na konkretne wartości. Została również dodana kolumna do oryginalnych danych w celu zamiany kodu stanu ('state') na pełną nazwę.

In [ ]:
import folium
from folium import plugins
import pandas as pd
import numpy as np
from folium.plugins import HeatMapWithTime, MarkerCluster

df = pd.read_csv('archive/historical-tornado-tracks.csv', sep=';')
do_usuniecia = ['AK', 'HI', 'PR', 'VI']
df['state'] = df['state'].str.strip()
df = df[~df['state'].isin(do_usuniecia)]

state_areas = {
    'AL': 50645, 'AK': 570641, 'AZ': 113594, 'AR': 52035, 'CA': 155779,
    'CO': 103642, 'CT': 4842, 'DE': 1949, 'FL': 53625, 'GA': 57513,
    'HI': 6423, 'ID': 82643, 'IL': 55519, 'IN': 35826, 'IA': 55857,
    'KS': 81759, 'KY': 39486, 'LA': 43204, 'ME': 30843, 'MD': 9707,
    'MA': 7800, 'MI': 56539, 'MN': 79627, 'MS': 46923, 'MO': 68742,
    'MT': 145546, 'NE': 76824, 'NV': 109781, 'NH': 8953, 'NJ': 7354,
    'NM': 121298, 'NY': 47126, 'NC': 48618, 'ND': 69001, 'OH': 40861,
    'OK': 68595, 'OR': 95988, 'PA': 44743, 'RI': 1034, 'SC': 30061,
    'SD': 75811, 'TN': 41235, 'TX': 261232, 'UT': 82170, 'VT': 9217,
    'VA': 39490, 'WA': 66456, 'WV': 24038, 'WI': 54158, 'WY': 97093,
    'DC': 61
}

state_names = {
    'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas', 'CA': 'California',
    'CO': 'Colorado', 'CT': 'Connecticut', 'DE': 'Delaware', 'FL': 'Florida', 'GA': 'Georgia',
    'HI': 'Hawaii', 'ID': 'Idaho', 'IL': 'Illinois', 'IN': 'Indiana', 'IA': 'Iowa',
    'KS': 'Kansas', 'KY': 'Kentucky', 'LA': 'Louisiana', 'ME': 'Maine', 'MD': 'Maryland',
    'MA': 'Massachusetts', 'MI': 'Michigan', 'MN': 'Minnesota', 'MS': 'Mississippi', 'MO': 'Missouri',
    'MT': 'Montana', 'NE': 'Nebraska', 'NV': 'Nevada', 'NH': 'New Hampshire', 'NJ': 'New Jersey',
    'NM': 'New Mexico', 'NY': 'New York', 'NC': 'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio',
    'OK': 'Oklahoma', 'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island', 'SC': 'South Carolina',
    'SD': 'South Dakota', 'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah', 'VT': 'Vermont',
    'VA': 'Virginia', 'WA': 'Washington', 'WV': 'West Virginia', 'WI': 'Wisconsin', 'WY': 'Wyoming',
    'DC': 'District of Columbia'
}
df['state_full'] = df['state'].map(state_names)

def lock_map(m_obj):
    m_obj.options['zoomControl'] = False
    m_obj.options['dragging'] = False
    m_obj.options['scrollWheelZoom'] = False
    m_obj.options['doubleClickZoom'] = False
    m_obj.options['boxZoom'] = False
    m_obj.options['keyboard'] = False

df.head()

In [ ]:
df.tail()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe()

### Mapa 1: Wielowarstwowa Analiza Przesunięcia Centroidów (1950-2015)

**Co przedstawia wizualizacja?**
Mapa ukazuje ewolucję geograficznego "środka ciężkości" (centroidu) aktywności tornad w kolejnych dekadach.
* **Punkty:** Każdy punkt to średnia szerokość i długość geograficzna wszystkich tornad z danej dekady.
* **Kodowanie czasu:** Kolor **niebieski** oznacza okres historyczny (1950-1979), a **czerwony** okres współczesny (1980-2015).
* **Warstwy (Layers):** Interaktywne menu (prawy górny róg) pozwala filtrować dane wg siły: od tornad umiarkowanych (F1-F2) po katastrofalne (F3-F5).

**Wnioski i realizacja celu:**
Wizualizacja bezpośrednio weryfikuje hipotezę o **przesunięciu strefy zagrożenia na wschód**.
1.  Wyrysowana linia trendu wskazuje wyraźny dryf centroidu z obszaru Oklahomy w kierunku Tennessee i Alabamy.
2.  Możliwość przełączenia na warstwę **"Bardzo mocne (F3-F5)"** pozwala potwierdzić, że trend ten nie wynika jedynie ze wzrostu populacji (częstsze raportowanie słabych tornad), ale jest rzeczywistą zmianą klimatologiczną dotyczącą również najpotężniejszych żywiołów.

In [ ]:
# PRZYGOTWANIE DANYCH
df1 = df[['yr', 'slat', 'slon', 'mag']]

# Wybranie stosunkowo silnych tornad - zgodnie ze skalą Fujity EF1 - oznacza umiarkowane zniszczenia
data_groups = {
    "Wszystkie znaczące (F1-F5)": df[df['mag'] >= 1].copy(),
    "Umiarkowane (F1-F2)": df[(df['mag'] >= 1) & (df['mag'] <= 2)].copy(),
    "Bardzo mocne (F3-F5)": df[df['mag'] >= 3].copy()
}

# Centroidy dla każdej grupy
centroids_dict = {}
for name, data in data_groups.items():
    data['decade'] = (data['yr'] // 10) * 10
    grp_centroid = data.groupby('decade')[['slat', 'slon']].mean().reset_index()
    centroids_dict[name] = grp_centroid

# RYSOWANIE MAPY
base_centroids = centroids_dict["Wszystkie znaczące (F1-F5)"]
avg_lat = base_centroids['slat'].mean() + 0.08
avg_lon = base_centroids['slon'].mean() - 0.1


m1 = folium.Map(location=[avg_lat, avg_lon], zoom_start=10, tiles='cartodbpositron')

for name, centroids in centroids_dict.items():
    fg = folium.FeatureGroup(name=name, show=(name == "Wszystkie znaczące (F1-F5)"))

    points = []
    for idx, row in centroids.iterrows():
        decade = int(row['decade'])
        color = 'blue' if decade < 1980 else 'red'
        folium.CircleMarker(
            location=[row['slat'], row['slon']],
            radius=8,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=1,
            popup=f"<b>Warstwa: {name}</b><br>Dekada: {decade}s",
            tooltip=f"{name}: {decade}s"
        ).add_to(fg)

        if decade == 1950:
            folium.Marker(
                [row['slat'], row['slon']],
                icon=folium.Icon(color='blue', icon='play', prefix='fa'),
                tooltip=f"POCZĄTEK ANALIZY"
            ).add_to(fg)

        if decade == 2010:
            folium.Marker(
                [row['slat'], row['slon']],
                icon=folium.Icon(color='red', icon='stop', prefix='fa'),
                tooltip=f"KONIEC ANALIZY"
            ).add_to(fg)

        points.append([row['slat'], row['slon']])

    weight = 3
    folium.PolyLine(points, color="#333333", weight=weight, opacity=0.6, dash_array='5, 5').add_to(fg)
    fg.add_to(m1)


folium.LayerControl(position='topright', collapsed=False).add_to(m1)

legend_html = '''
<div style="
    position: fixed;
    bottom: 50px; left: 50px; width: 230px; height: 160px;
    border:2px solid grey; z-index:9999; font-size:14px;
    background-color:white; opacity: 0.9;
    padding: 10px; border-radius: 10px;
    box-shadow: 3px 3px 5px rgba(0,0,0,0.3);
    ">
    <b>Legenda: Oś Czasu</b><br>
    <hr style="margin: 5px 0;">
    <i class="fa fa-circle" style="color:blue"></i> Lata 1950 - 1979<br>
    <small>(Starsze dane)</small><br>

    <i class="fa fa-circle" style="color:red"></i> Lata 1980 - 2015<br>
    <small>(Nowsze dane)</small><br>

    <i class="fa fa-minus" style="color:black; font-weight:bold"></i> Ścieżka przesunięcia<br>
    <br>
</div>
'''

title_html = '''
     <h3 align="center" style="font-size:32px;"><b>Analiza wielowarstwowa: Przesunięcie centroidów tornad</b></h3>
     '''

m1.get_root().html.add_child(folium.Element(legend_html))
m1.get_root().html.add_child(folium.Element(title_html))


m1.save('mapa1_layers_final.html')
#m1

### Mapa 2: Lokalizacja Tornad Katastroficznych (EF4-EF5)

**Co przedstawia wizualizacja?**
Mapa punktowa skupiająca się wyłącznie na zjawiskach o niszczycielskiej sile - tornadach kategorii **EF4** (pomarańczowe) i **EF5** (czerwone).
* **Technika Klastrowania (Marker Cluster):** Zastosowano grupowanie punktów, aby zachować czytelność mapy przy oddaleniu i uwidocznić "hotspoty" (regiony o największym zagęszczeniu katastrof).
* **Interaktywne Karty:** Po kliknięciu w punkt wyświetla się estetyczna karta ze szczegółowymi danymi: datą, pełną nazwą stanu oraz liczbą ofiar śmiertelnych i rannych.

**Wnioski i realizacja celu:**
Mapa realizuje cel **"Intensywność zjawisk"** oraz **"Analizę bezpieczeństwa"**.
1.  Wizualizacja obala mit, że najsilniejsze tornada występują tylko na Wielkich Równinach. Wyraźne klastry EF4/EF5 widoczne są w stanach południowo-wschodnich (Alabama, Mississippi).
2.  Dostęp do danych o ofiarach (w popupach) pozwala zauważyć korelację między lokalizacją tornada a jego śmiertelnością – "Wielkie Bestie" w gęsto zaludnionym "Dixie Alley" często generują więcej ofiar niż podobne zjawiska na otwartych równinach.

In [ ]:
df_extreme = df[(df['mag'] == 4) | (df['mag'] == 5)]
css_popup = """
<style>
.leaflet-popup-content-wrapper, .leaflet-popup-tip {
    background: rgba(255, 255, 255, 0.9) !important;
    box-shadow: 0 5px 15px rgba(0,0,0,0.3) !important;
    border-radius: 8px !important;
}
.leaflet-popup-content {
    margin: 0 !important;
    width: auto !important;
}
</style>
"""
m2 = folium.Map(location=[38, -95], zoom_start=5, tiles='cartodbpositron')

marker_cluster = MarkerCluster(
    disableClusteringAtZoom=9,
    maxClusterRadius=45
).add_to(m2)

for idx, row in df_extreme.iterrows():
    color = 'red' if row['mag'] == 5 else 'orange'
    header_bg = 'red' if row['mag'] == 5 else 'orange'

    popup_text = f"""
    <div style="font-family: 'Segoe UI', Arial, sans-serif; min-width: 220px; overflow: hidden; border-radius: 8px;">

        <div style="background-color: {header_bg}; color: white; padding: 10px 15px;">
            <h4 style="margin: 0; font-size: 16px; text-transform: uppercase; letter-spacing: 1px;">
                Tornado EF{row['mag']}
            </h4>
        </div>

        <div style="padding: 15px;">
            <table style="width: 100%; border-collapse: collapse; font-size: 13px; color: #333;">
                <tr style="border-bottom: 1px solid #eee;">
                    <td style="padding: 5px 0; font-weight: bold; color: #555;">Data:</td>
                    <td style="padding: 5px 0; text-align: right;">{row['yr']}-{row['mo']}-{row['dy']}</td>
                </tr>
                <tr style="border-bottom: 1px solid #eee;">
                    <td style="padding: 5px 0; font-weight: bold; color: #555;">Stan:</td>
                    <td style="padding: 5px 0; text-align: right;">{row['state_full']}</td>
                </tr>
                <tr style="border-bottom: 1px solid #eee;">
                    <td style="padding: 5px 0; font-weight: bold; color: #555;">Ofiary śmiertelne:</td>
                    <td style="padding: 5px 0; text-align: right; color: #d32f2f;">{row['fat']}</td>
                </tr>
                <tr>
                    <td style="padding: 5px 0; font-weight: bold; color: #555;">Ranni:</td>
                    <td style="padding: 5px 0; text-align: right;">{row['inj']}</td>
                </tr>
            </table>
        </div>
    </div>
    """

    folium.CircleMarker(
        location=[row['slat'], row['slon']],
        radius=10,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.8,
        popup=folium.Popup(popup_text, max_width=200),
        tooltip=f"EF{row['mag']} ({row['yr']}) - Kliknij po szczegóły"
    ).add_to(marker_cluster)


title_html = '''
     <h3 align="center" style="color: black; font-size:32px"><b>Lokalizacja tornad katastroficznych (EF4-EF5)</b></h3>
     '''
m2.get_root().html.add_child(folium.Element(title_html))

m2.save('mapa2_extreme_tornadoes.html')
#m2

### Mapa 3: Dualna Analiza Ryzyka – Dzień vs Noc

**Co przedstawia wizualizacja?**
Zaawansowane zestawienie dwóch zsynchronizowanych map (Side-by-side), porównujących skutki tornad w zależności od pory dnia. Skala kolorystyczna reprezentuje łączną liczbę poszkodowanych (ofiary śmiertelne + ranni).
* **Panel Lewy (Dzień ☀️):** Mapa na jasnym podkładzie, prezentująca statystyki dla zjawisk dziennych (z uwzględnieniem zmiennej długości dnia w różnych porach roku).
* **Panel Prawy (Noc 🌙):** Mapa na ciemnym podkładzie, dedykowana zjawiskom nocnym.
* **Interakcja:** Mapy są ze sobą powiązane - przesunięcie lub przybliżenie jednej automatycznie steruje drugą, co pozwala na precyzyjne porównanie tych samych stanów. Najechanie na stan wyświetla szczegółowe statystyki dla obu pór dnia jednocześnie.

**Wnioski i realizacja celu:**
Wizualizacja realizuje cel **"Czynnik dobowy"** i dostarcza kluczowych informacji o **bezpieczeństwie**:
1. **Dotkliwość tornad w nocy:** Można zauważyć, że w nocy najbardziej narażonym obszarem jest tzw. "Dixie Alley", co pokazuje, że mieszkańcy tych rejonów powinni być szczególnie czujni w ciągu nocy, a systemy ostrzegania powinny być wzmocnione w tych rejonach.
2. **Można by było dodać jeszcze jakiś wniosek**

In [ ]:
# 1. PRZYGOTOWANIE DANYCH
df_dual_map = df[['state_full', 'mo', 'time', 'fat', 'inj']].copy()
df_dual_map['datetime_obj'] = pd.to_datetime(df_dual_map['time'], format='%H:%M:%S', errors='coerce')
df_dual_map['hour'] = df_dual_map['datetime_obj'].dt.hour
df_dual_map['casualty_count'] = df_dual_map['fat'] + df['inj']

def is_daytime(row):
    m = row['mo']
    h = row['hour']
    if pd.isna(h): return False
    if m in [11, 12, 1, 2]: return 8 <= h < 17
    elif m in [5, 6, 7, 8]: return 6 <= h < 21
    else: return 7 <= h < 19

df_dual_map['is_day'] = df_dual_map.apply(is_daytime, axis=1)

state_stats = df_dual_map.groupby(['state_full', 'is_day'])['casualty_count'].sum().unstack(fill_value=0)
state_stats.columns = ['Night', 'Day']
state_stats = state_stats.reset_index()

max_val = max(state_stats['Day'].max(), state_stats['Night'].max())
bins = list(np.linspace(0, max_val, 8))

# 2. INICJALIZACJA DUAL MAP
m_dual = plugins.DualMap(
    location=[38, -95],
    zoom_start=4,
    layout='horizontal',
    tiles=None
)

# CSS: Usuwamy ramkę przy kliknięciu (focus)
m_dual.get_root().html.add_child(folium.Element("""
<style>
    path:focus { outline: none !important;
    }

    .leaflet-tooltip {
        background-color: rgba(255, 255, 255, 0.9);
        border: 2px solid #555;
        border-radius: 10px;
        box-shadow: 4px 4px 10px rgba(0,0,0,0.5);
        font-family: 'Helvetica', sans-serif;
        font-size: 14px;
        padding: 0px !important;
        overflow: hidden;
    }

    .leaflet-tooltip table {
        border-collapse: collapse;
        margin: 0;
    }

    .leaflet-tooltip tr {
        border-bottom: 1px solid #ddd;
    }
    .leaflet-tooltip tr:last-child {
        border-bottom: none;
    }
    .leaflet-tooltip td {
        padding: 8px 12px;
    }

    .leaflet-tooltip td:first-child {
        font-weight: bold;
        color: #555;
</style>
"""))
lock_map(m_dual.m1)
lock_map(m_dual.m2)

# Tła
folium.TileLayer('cartodbpositron').add_to(m_dual.m1)
folium.TileLayer('cartodbdarkmatter').add_to(m_dual.m2)


# 3. PRZYGOTOWANIE WARSTW I TOOLTIPÓW
us_states_url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data/us-states.json'

# Funkcja pomocnicza do wstrzykiwania danych do GeoJSON
# Dzięki temu w tooltipie dla DNIA będziemy mieli też dane dla NOCY (i odwrotnie)
def inject_data_to_geojson(choropleth_obj, stats_df):
    for feature in choropleth_obj.geojson.data['features']:
        state_name = feature['properties']['name']
        row = stats_df[stats_df['state_full'] == state_name]

        # Pobieramy obie wartości
        val_day = int(row['Day'].values[0]) if not row.empty else 0
        val_night = int(row['Night'].values[0]) if not row.empty else 0

        # Zapisujemy je w properties, żeby tooltip miał do nich dostęp
        feature['properties']['val_day'] = val_day
        feature['properties']['val_night'] = val_night

# --- LEWA STRONA (DZIEŃ) ---
cp_day = folium.Choropleth(
    geo_data=us_states_url,
    data=state_stats,
    columns=['state_full', 'Day'],
    key_on='feature.properties.name',
    fill_color='YlOrRd',
    fill_opacity=1.0,
    line_opacity=0.2,
    nan_fill_opacity=0.0,
    bins=bins,
    highlight=True
)


inject_data_to_geojson(cp_day, state_stats)

cp_day.geojson.add_child(
    folium.features.GeoJsonTooltip(
        fields=['name', 'val_day', 'val_night'],
        aliases=['Stan:', 'Poszkodowani w DZIEŃ:', 'Poszkodowani w NOCY:'],
        style=("background-color: white; color: black; font-family: arial")
    )
)
for key in list(cp_day._children):
    if key.startswith('color_map'): del cp_day._children[key]
cp_day.add_to(m_dual.m1)


cp_night = folium.Choropleth(
    geo_data=us_states_url,
    data=state_stats,
    columns=['state_full', 'Night'],
    key_on='feature.properties.name',
    fill_color='YlOrRd',
    fill_opacity=1.0,
    line_opacity=0.2,
    nan_fill_opacity=0.0,
    bins=bins,
    highlight=True
)

inject_data_to_geojson(cp_night, state_stats)

cp_night.geojson.add_child(
    folium.features.GeoJsonTooltip(
        fields=['name', 'val_night', 'val_day'],
        aliases=['Stan:', 'Poszkodowani w NOCY:', 'Poszkodowani w DZIEŃ:'],
        style=("background-color: black; color: black; font-family: arial")
    )
)
for key in list(cp_night._children):
    if key.startswith('color_map'): del cp_night._children[key]
cp_night.add_to(m_dual.m2)


legend_html = f'''
<div style="
    position: fixed;
    bottom: 30px;
    left: 50%;
    transform: translateX(-50%);
    z-index: 9999;
    background-color: rgba(255,255,255,0.9);
    padding: 15px;
    border-radius: 50px;
    border: 2px solid #555;
    box-shadow: 0 0 15px rgba(0,0,0,0.4);
    text-align: center;
    font-family: Arial, sans-serif;
    min-width: 300px;
">
    <div style="font-weight: bold; margin-bottom: 5px; font-size: 14px;">Liczba poszkodowanych (0 - {int(max_val)})</div>

    <div style="
        width: 100%;
        height: 20px;
        background: linear-gradient(to right, #ffffb2, #fecc5c, #fd8d3c, #f03b20, #bd0026);
        border-radius: 10px;
        border: 1px solid #ccc;
    "></div>

    <div style="display: flex; justify-content: space-between; font-size: 12px; margin-top: 5px;">
        <span>Mało</span>
        <span>Dużo</span>
    </div>
</div>
'''
m_dual.get_root().html.add_child(folium.Element(legend_html))


# 5. NAGŁÓWKI
title_day_html = '''
<div style="position: fixed; top: 10px; left: 10%; z-index:999;
     background-color: rgba(255,255,255,0.9); padding: 10px; border-radius: 8px; border: 1px solid #ccc;">
     <h3 style="margin:0; color:black;">Porównanie ilości ofiar i rannych w dzień ☀️</h3>
</div>
'''
title_night_html = '''
<div style="position: fixed; top: 10px; right: 10%; z-index:999;
     background-color: rgba(0,0,0,0.8); padding: 10px; border-radius: 8px; border: 1px solid #666;">
     <h3 style="margin:0; color:white;">Porównanie ilości ofiar i rannych w nocy 🌙</h3>
</div>
'''
m_dual.m1.get_root().html.add_child(folium.Element(title_day_html))
m_dual.m2.get_root().html.add_child(folium.Element(title_night_html))


m_dual.save('mapa3_dual_day_night.html')


### Mapa 4: Występowanie tornad w USA w latach (1950-2015).

**Co przedstawia wizualizacja?**
Dynamiczna animacja (HeatMap With Time) ukazująca zmiany gęstości występowania tornad w interwałach 5-letnich.
* **System Wagowy:** Zastosowano nieliniowe ważenie punktów (`(mag + 1)^2`), dzięki czemu silne tornada (EF3-EF5) generują znacznie intensywniejszą "plamę ciepła" niż słabe. Pozwala to odfiltrować szum informacyjny (tysiące niegroźnych tornad EF0) i skupić wzrok na obszarach realnego zagrożenia.
* **Oś Czasu:** Odtwarzacz na dole mapy pozwala automatycznie prześledzić historię lub zatrzymać się na wybranym okresie (np. lata 2010-2015).

**Wnioski i realizacja celu:**
Wizualizacja realizuje cel **"Wizualizację trendów"** oraz **"Zmiany w lokalizacji tornad"**.
1.  **Pulsacja aktywności:** Mapa doskonale uwidacznia okresy wzmożonej aktywności.
2.  **Ewolucja przestrzenna:** Oglądając animację, można dostrzec, jak "serce" aktywności tornad, początkowo skupione w Teksasie i Oklahomie, w późniejszych okresach coraz częściej rozlewa się na wschód (Alabama, Tennessee), co wizualnie potwierdza tezę o rozszerzaniu się strefy zagrożenia.

In [ ]:
df_heat = df[['yr', 'slat', 'slon', 'mag']].copy()

df_heat = df_heat[df_heat['mag'] >= 0]

df_heat['weight'] = (df_heat['mag'] + 1) ** 2
df_heat['norm_weight'] = df_heat['weight'] / df_heat['weight'].max()

df_heat['period_start'] = (df_heat['yr'] // 5) * 5
df_heat.loc[df_heat['yr'] == 2015, 'period_start'] = 2010

periods = sorted(df_heat['period_start'].unique())

data_list = []
index_labels = []

for p_start in periods:
    if p_start + 5 >= 2015:
        p_end = '-2015'
    else:
        p_end = '-'
        p_end += str(p_start + 4)

    label = f"{p_start}{p_end}"
    index_labels.append(label)

    mask = df_heat['period_start'] == p_start
    period_data = df_heat[mask][['slat', 'slon', 'norm_weight']].values.tolist()

    data_list.append(period_data)

m_heat = folium.Map(location=[38, -95], zoom_start=5, tiles='cartodbpositron')


HeatMapWithTime(
    data_list,
    index=index_labels,
    auto_play=True,
    radius=20,
    min_opacity=0.4,
    max_opacity=1.0,
    use_local_extrema=False,
    position='bottomright'
).add_to(m_heat)

title_html = '''
     <h3 align="center" style="color: black; font-size:32px"><b>Występowanie Tornad w USA w latach (1950 - 2015)</b></h3>
     '''
m_heat.get_root().html.add_child(folium.Element(title_html))

m_heat.save('mapa4_heatmap_animated.html')


### Mapa 5: Kartogram Intensywności - Analiza Porównawcza Stanów (1950-1980 vs 1981-2015)

**Co przedstawia wizualizacja?**
Interaktywny kartogram pozwalający na bezpośrednie porównanie dwóch epok klimatycznych: "Starej" (1950-1980) i "Nowej" (1981-2015). Mapa oferuje dwie perspektywy analityczne:
* Całkowity Potencjał Zniszczeń:** Sumaryczna siła tornad w danym stanie (ważona kwadratem magnitudy, aby uwypuklić zjawiska ekstremalne).
* Zagrożenie Znormalizowane:** Kluczowy wskaźnik pokazujący intensywność zjawisk w przeliczeniu na milę kwadratową. Eliminuje to błąd poznawczy wynikający z wielkości stanów (np. ogromny Teksas vs małe stany wschodnie).

**Wnioski i realizacja celu:**
Jest to **narzędzie weryfikujące** główną hipotezę projektu.
1. **Porównując warstwę *1981-2015* z okresem wcześniejszym**, obserwujemy duży wzrost intensywności tornad we wschodnich obszarach USA (przesunięcie w stronę "Dixie Alley").
2. **Mapa udowadnia, że choć Teksas dominuje liczbowo (bezwzględnie)**, to największe zagęszczenie silnych tornad w ostatnich dekadach przeniosło się do regionu "Dixie Alley", co czyni ten obszar nowym, najgroźniejszym frontem burzowym w USA.

In [ ]:
df_cart = df[['yr', 'slat', 'slon', 'mag', 'state','state_full', 'fat', 'inj']].copy()
df_cart['weighted_score'] = (df['mag']) ** 2

df_old = df_cart[(df_cart['yr'] >= 1950) & (df_cart['yr'] <= 1980)]
df_new = df_cart[(df_cart['yr'] >= 1981) & (df_cart['yr'] <= 2015)]

def prepare_state_data(df_subset):
    data = df_subset.groupby(['state','state_full']).agg({
        'weighted_score': 'sum',
        'fat': 'sum',
        'inj': 'sum',
        'mag': 'count'
    }).reset_index()

    mag_counts = df_subset.groupby(['state', 'mag']).size().unstack(fill_value=0)
    mag_counts.columns = [f'F{int(c)}' for c in mag_counts.columns]
    data = data.merge(mag_counts, on='state', how='left')
    cols_to_fill = [c for c in data.columns if c.startswith('F')]
    data[cols_to_fill] = data[cols_to_fill].fillna(0)

    data['area'] = data['state'].map(state_areas)
    data['state_full_name'] = data['state_full']
    data['score_per_area'] = (data['weighted_score'] / data['area']) * 1000
    data.rename(columns={'mag': 'count'}, inplace=True)
    return data

state_data_old = prepare_state_data(df_old)
state_data_new = prepare_state_data(df_new)



us_states_url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data/us-states.json'
map_cart = folium.Map(location=[38, -96], zoom_start=5, tiles='cartodbpositron')

lock_map(map_cart)

map_cart.get_root().html.add_child(folium.Element("""
<style>
    path:focus {
    outline: none !important;
    }

    .legend .tick {
        display: none;
    }

    .custom-legend {
        position: fixed;
        bottom: 30px;
        left: 30px;
        width: 250px;
        background-color: rgba(255, 255, 255, 0.9);
        border: 2px solid #555;
        border-radius: 8px;
        padding: 15px;
        font-family: 'Helvetica', sans-serif;
        z-index: 9999;
        box-shadow: 4px 4px 10px rgba(0,0,0,0.3);
    }
    .legend-title {
        font-weight: bold;
        font-size: 16px;
        margin-bottom: 8px;
        color: #333;
    }
    .legend-scale {
        display: flex;
        flex-direction: column;
        gap: 5px;
    }
    .legend-item {
        display: flex;
        align-items: center;
        font-size: 14px;
    }
    .legend-box {
        width: 20px;
        height: 20px;
        margin-right: 10px;
        border: 1px solid #999;
    }
    .leaflet-tooltip {
        background-color: rgba(255, 255, 255, 0.9);
        border: 2px solid #555;
        border-radius: 10px;
        box-shadow: 4px 4px 10px rgba(0,0,0,0.5);
        font-family: 'Helvetica', sans-serif;
        font-size: 14px;
        padding: 0px !important;
        overflow: hidden;
    }

    .leaflet-tooltip table {
        border-collapse: collapse;
        margin: 0;
    }

    .leaflet-tooltip tr {
        border-bottom: 1px solid #ddd;
    }
    .leaflet-tooltip tr:last-child {
        border-bottom: none;
    }
    .leaflet-tooltip td {
        padding: 8px 12px;
    }

    .leaflet-tooltip td:first-child {
        font-weight: bold;
        color: #555;
    }
    .leaflet-control-layers {
        position: absolute !important;
        top: 10px !important;
        right: 10px !important;
    }

</style>
"""))

#def dodaj_warstwe(data, name, column, color, legend_title=None, show=True):
def dodaj_warstwe(data, name, column, color, show=True):
    cp = folium.Choropleth(
        geo_data=us_states_url,
        name=name,
        data=data,
        columns=['state', column],
        key_on='feature.id',
        fill_color=color,
        fill_opacity=1.0,
        line_opacity=0.2,
        nan_fill_opacity=0,
        highlight=True,
        legend_name=None,
        show=show
    )

    for feature in cp.geojson.data['features']:
        state_id = feature['id']
        row = data[data['state'] == state_id]

        if not row.empty:
            feature['properties']['p_name'] = str(row.iloc[0]['state_full'])
            feature['properties']['p_fat'] = int(row.iloc[0]['fat'])
            feature['properties']['p_inj'] = int(row.iloc[0]['inj'])
            feature['properties']['p_count'] = int(row.iloc[0]['count'])
            breakdown_parts = []
            for i in range(6):
                col_name = f'F{i}'
                if col_name in row.columns:
                    val = int(row.iloc[0][col_name])
                    breakdown_parts.append(f"{col_name}: {val}")
            feature['properties']['p_breakdown'] = ", ".join(breakdown_parts)
            if column == 'score_per_area':
                feature['properties'][column] = f"{row.iloc[0][column]:.2f}%"
            else:
                feature['properties'][column] = int(row.iloc[0][column])
        else:
            feature['properties']['p_name'] = 'Brak danych'
            feature['properties']['p_fat'] = 'Brak danych'
            feature['properties']['p_inj'] = 'Brak danych'
            feature['properties']['p_count'] = 'Brak danych'
            feature['properties']['p_breakdown'] = 'Brak danych'
            feature['properties'][column] = 'Brak danych'

    cp.geojson.add_child(
        folium.features.GeoJsonTooltip(
            fields=['p_name', 'p_fat', 'p_inj', 'p_count', 'p_breakdown',column],
            aliases=['Stan:', 'Ofiary śmiertelne:', 'Ranni:', 'Liczba tornad:','Siła tornad:', 'Waga tornad:'],
            style=("background-color: white; color: #333333; font-family: arial; font-size: 16px; padding: 10px;")
        )
    )

    for key in list(cp._children):
        if key.startswith('color_map'):
            del cp._children[key]
    # if legend_title is None:
    #     for key in list(cp._children):
    #         if key.startswith('color_map'):
    #             del cp._children[key]

    cp.add_to(map_cart)

dodaj_warstwe(state_data_old, '1950-1980', 'weighted_score', 'YlOrRd',
              #legend_title='Suma siły tornad (pkt)',
              show=True)

dodaj_warstwe(state_data_new, '1981-2015', 'weighted_score', 'YlOrRd',
              #legend_title=None,
              show=False)

dodaj_warstwe(state_data_old, '1950-1980 w porównaniu do powierzchni stanu', 'score_per_area', 'BuPu',
              #legend_title='Wskaźnik zagrożenia na milę kw.',
              show=False)

dodaj_warstwe(state_data_new, '1981-2015 w porównaniu do powierzchni stanu', 'score_per_area', 'BuPu',
              #legend_title=None,
              show=False)

folium.LayerControl(collapsed=False).add_to(map_cart)
title_html = '''
     <h3 align="center" style="font-size:32px;"><b>Intensywność występowania tornad w poszczególnych stanach przed i po 1980</b></h3>
     '''
legend_html = '''
<div class="custom-legend">
    <div class="legend-title">Suma siły tornad (YlOrRd)</div>
    <div class="legend-scale">
        <div class="legend-item"><span class="legend-box" style="background:#FFFFB2;"></span> Niska</div>
        <div class="legend-item"><span class="legend-box" style="background:#FECC5C;"></span> Średnia</div>
        <div class="legend-item"><span class="legend-box" style="background:#FD8D3C;"></span> Wysoka</div>
        <div class="legend-item"><span class="legend-box" style="background:#E31A1C;"></span> Bardzo wysoka</div>
    </div>
    <br>
    <div class="legend-title">Zagrożenie na milę (BuPu)</div>
    <div class="legend-scale">
        <div class="legend-item"><span class="legend-box" style="background:#E0ECF4;"></span> Minimalne</div>
        <div class="legend-item"><span class="legend-box" style="background:#9EBCDA;"></span> Umiarkowane</div>
        <div class="legend-item"><span class="legend-box" style="background:#8856A7;"></span> Znaczne</div>
    </div>
</div>
'''

map_cart.get_root().html.add_child(folium.Element(title_html))
map_cart.get_root().html.add_child(folium.Element(legend_html))
map_cart.save('mapa5_kartogram_stany.html')

Wnioski
